In [4]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras
from keras.applications.vgg16 import preprocess_input as vgg16_preproc
from skimage.transform import resize
from keras.models import model_from_json

Using TensorFlow backend.


In [16]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename):  
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    
    if ds.Modality != 'DX':
        print(f'Image rejected. Reason: modality is {ds.Modality}.\n')
        return None, ds.StudyDescription
    
    if ds.BodyPartExamined != 'CHEST':
        print(f'Image rejected. Reason: body part is {ds.BodyPartExamined}\n')
        return None, ds.StudyDescription
    
    if ds.PatientPosition not in ['AP', 'PA']:
        print(f'Image rejected. Reason: position is {ds.PatientPosition}.\n' )
        return None, ds.StudyDescription
    
    img = ds.pixel_array
    return img, ds.StudyDescription
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img, img_mean, img_std, img_size): 
    proc_img = img / 255.0
    proc_img = resize(proc_img, (img_size[1], img_size[2]), anti_aliasing=False)
    proc_img = proc_img.reshape((img_size[0], img_size[1], img_size[2], 1))
    proc_img = np.repeat(proc_img, img_size[3], axis=3)
    proc_img = vgg16_preproc(proc_img)
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    json_file = open(model_path, 'r')
    model_json = json_file.read()
    json_file.close()
    model = model_from_json(model_json)
    model.load_weights(weight_path)   
    #model = model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    prob = model.predict(img)
    pred= prob > thresh
    return pred[0][0]

In [17]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'model_vgg.json'
weight_path = 'xray_class_my_model_vgg.best1.hdf5'

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = -1 # loads the mean image value they used during training preprocessing
img_std = -1 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.39 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img, study_description = check_dicom(i)
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model, img_proc, thresh)
    print(f'Prediction: {"No pneumonia" if pred == False else "Pneumonia"}')
    print(f'Actual: {study_description}\n')
    

Load file test1.dcm ...
Prediction: No pneumonia
Actual: No Finding

Load file test2.dcm ...
Prediction: No pneumonia
Actual: Cardiomegaly

Load file test3.dcm ...
Prediction: No pneumonia
Actual: Effusion

Load file test4.dcm ...
Image rejected. Reason: body part is RIBCAGE

Load file test5.dcm ...
Image rejected. Reason: modality is CT.

Load file test6.dcm ...
Image rejected. Reason: position is XX.

